# Import Libraries

In [1]:
from swarmintelligence import SlimeMouldAlgorithm, Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
optimal_params = pickle.load(open('results/hyperparameter_tuning/SMA/optimal_params.pkl', 'rb'))
optimal_params

{'pop_size': 25, 'max_iteration': 100, 'z': 0.3}

# Evaluation

In [5]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
n_runs = 30

In [6]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_g = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_b = SlimeMouldAlgorithm(
                        k=threshold,
                        slimeSize=optimal_params['pop_size'],
                        maxIteration=optimal_params['max_iteration'],
                        z=optimal_params['z'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
919.3612020672858  |  2295.0456642907693  |  1712.6186484607524
921.6332246278098  |  2326.0224072123797  |  1735.9842841954512
842.7298622665826  |  2384.975071638772  |  1724.6467032245337
848.5705315769846  |  2403.8730706504725  |  1777.610177161595
888.6535172816343  |  2213.376139082271  |  1767.567179530629
857.9612528761963  |  2386.9880320951206  |  1781.6311509136794
862.5782392451766  |  2304.8658836161976  |  1672.5594395026346
798.6767426338963  |  2419.5312021319664  |  1762.0869337990018
923.4378080746336  |  2351.571079617748  |  1733.6932786620237
925.3534887091919  |  2366.5635005485237  |  1777.6166670307869
895.8953307393903  |  2367.785612981216  |  1623.4504684535555
874.918930393637  |  2414.8125892606186  |  1775.143522748109
892.2646870435059  |  2360.2615087538206  |  1739.0440799690064
913.2008131403503  |  2427.19030130181  |  1737.213431888396
885.2795027247748  |  2397.611589825149  |  1785.7334329060045
927.099247847054

992.5117574694788  |  2265.0223696726343  |  2104.4509382643073
943.8503060898321  |  1896.5496498740133  |  2177.8577726093886
978.9737043009306  |  2356.515603391652  |  2151.913029005977
768.3288539514106  |  2194.3288294278054  |  2180.9741786928735
1000.8080132902717  |  2087.094569003156  |  2166.9735460962124
947.4419646572089  |  2328.285253254169  |  2180.9038541370096
985.440575365005  |  2312.834774428351  |  2149.4929649651367
941.1678108720666  |  2048.220782555928  |  2148.1104217216775
998.5556204539656  |  2192.3684261524977  |  2143.3173043160677
910.7910415473259  |  2352.3098665551424  |  2127.8056644268877
912.9845444857469  |  2040.4903308388803  |  2163.3602604973717
869.8067873679345  |  2275.7475551094813  |  2007.154646246045
787.1391223888781  |  2284.3576441148625  |  2152.068170717734
913.9456514860408  |  2164.6209104539084  |  2154.425381593631
902.0552200042985  |  2364.343031271102  |  1948.3744904435544
989.4804985747369  |  2356.0893643386553  |  2151.

2692.6502374596253  |  2502.3509390072404  |  2933.7522823705403
2882.354950793616  |  2455.5310929997313  |  2806.161902943472
2740.940277685854  |  2768.9340283109896  |  2871.1102474393783
2520.045641959241  |  2941.8580088002514  |  2838.400568998732
2943.849255660367  |  2716.4523587425842  |  2945.6770927250373
2749.461376715393  |  2940.742741011624  |  2891.030342802191
2620.4409113971624  |  2903.210312785648  |  2796.2523093064283
2781.1022283504863  |  2848.377481226381  |  2884.8618622856243
2923.307314140611  |  2566.823655032555  |  2745.3978771607553
2848.1197534834664  |  2864.0111179862392  |  2499.238810585076
2813.907864058039  |  2821.127563462098  |  2575.992409835033
2879.8759349864  |  2641.808439619029  |  2802.395055819344
2793.849704980128  |  2895.5002136342246  |  2928.6746402284716
2874.1565631851145  |  2925.7470531677154  |  2902.4324279606108
2789.049620761556  |  2891.127856310819  |  2930.525213951548
2733.4069869421833  |  2656.5469118814735  |  2582.

2907.818646346228  |  1900.3793092458563  |  2601.229696025377
3344.126032339909  |  1770.6170223374047  |  2514.162972095282
3357.9518248374443  |  1933.3883618482498  |  2552.6404500063295
3224.2432180471965  |  1695.9999215286452  |  2578.768915733609
3329.38137979376  |  1948.773639533233  |  2338.0699982503024
3113.6469884236158  |  1751.4704151428896  |  2507.8177226184134
3351.7104427839395  |  1894.5851187921535  |  2415.5852726084786
3355.540417883092  |  2004.642471593289  |  2517.515284151557
3016.13015111184  |  1692.3931619701357  |  2628.671446203118
3068.1027829774234  |  1932.4076051960074  |  2591.957830204466
3106.604239089943  |  1882.2769042225787  |  2415.8741309391603
3105.647295507656  |  1936.0222978109787  |  2548.7392804520528
2998.406912780568  |  1962.2796897502994  |  2534.6826640603567
3091.3528394765726  |  1701.4890564784614  |  2632.229095189885
3243.6688784554494  |  1958.599339608558  |  2323.154302812806
3252.0023721615576  |  1992.8290382351051  |  

1547.8860236450737  |  5110.246303367334  |  1729.6217157590936
1535.6099278193326  |  5169.389573976843  |  1660.970723499897
1615.387768711032  |  4924.432162076688  |  1530.7602676895222
1569.3847429273433  |  5115.83687844939  |  1466.2304441034362
1636.2457592016613  |  5067.077237897887  |  1619.5582956723144
1167.37369701517  |  5199.838700604156  |  1593.9101049168553
1468.6765645334872  |  5200.772151862131  |  1662.4816189482583
1236.2752765071004  |  5026.257236115027  |  1671.4945215317834
1648.4038888347095  |  4982.874099914647  |  1713.3458474847002
1608.010723484117  |  5055.139850999893  |  1498.5866885786759
1654.4138943020253  |  5169.0027649799285  |  1707.9394254438205
1369.443710901311  |  5070.337649748494  |  1692.8025929407
1377.7977805844907  |  5188.72996353151  |  1695.5390537687867
1388.4470387451552  |  5004.39067098793  |  1683.2375274892015
1366.5640816720497  |  4878.087799208897  |  1498.3527291842684
1396.0426833145902  |  5140.00141176246  |  1758.33

5504.427908949125  |  5533.763613017287  |  1555.7008922494306
5624.862635139636  |  5527.991998020146  |  1559.5747552654975
5537.6565143769385  |  5296.297527177993  |  1527.6193624844373
5603.216671061884  |  5410.97735271023  |  1580.5132733848395
5459.297551577876  |  5600.912911673536  |  1587.141845065334
5626.633770491724  |  5473.238579900024  |  1499.388525667644
5540.310854344394  |  5510.723401233599  |  1460.3347128476053
5550.11752401611  |  5549.430823502358  |  1428.181729138807
5469.040273688174  |  5395.011526715497  |  1501.0532896305701
5629.098664356086  |  5372.695148477556  |  1504.5138687567237
5624.260614311881  |  5623.787434131724  |  1574.740507929318
5610.7099982047175  |  5618.125535133428  |  1430.5530950842576
5358.235648350723  |  5586.590716802566  |  1468.754111040168
5524.449065912098  |  5630.321003788622  |  1587.2153620060185
5392.284082387339  |  5222.447194833009  |  1576.7583385401772
5575.1130540040485  |  5259.995318244173  |  1449.4097876783

## Visualize Results using DataFrame

In [7]:
gwo_evaluation_results_df =pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[120, 173]","[98, 173]","[115, 180]",890.589809,2371.650768,1743.622864,...,"[[115, 255, 255, 255, 255, 255, 255, 255, 255,...","[919.3612020672858, 921.6332246278098, 842.729...","[2295.0456642907693, 2326.0224072123797, 2384....","[1712.6186484607524, 1735.9842841954512, 1724....","[2.384199857711792, 2.802081346511841, 2.37533...","[2008.5930773417156, 2350.960475921631, 2501.0...","[44.81733010055056, 48.48670411485638, 50.0104...","[15.101883993138914, 14.418350329907064, 14.14...","[0.7102264550468996, 0.7483391500478472, 0.738...","[0.9430088606654725, 0.9383241946804611, 0.934..."
1,AirplaneF16.tiff,3,otsu,max,"[85, 142, 186]","[56, 123, 184]","[74, 142, 189]",927.140018,2450.084303,1806.023968,...,"[[142, 255, 255, 255, 255, 255, 255, 255, 255,...","[914.8307420228217, 958.9920206765021, 934.949...","[2419.9555599306045, 2533.6525462786467, 2480....","[1789.2742872408883, 1849.4695647677995, 1836....","[4.180053234100342, 3.7882728576660156, 4.3039...","[1852.6823984781902, 1770.658526102702, 1936.5...","[43.0427972891887, 42.079193505849204, 44.0061...","[15.45279385375478, 15.649455458824328, 15.260...","[0.7788982559482084, 0.7718625850122885, 0.755...","[0.9512621468349144, 0.9552078431435685, 0.949..."
2,AirplaneF16.tiff,4,otsu,max,"[70, 126, 164, 197]","[27, 88, 143, 189]","[59, 119, 166, 195]",949.327218,2499.663982,1840.298653,...,"[[119, 255, 255, 255, 255, 255, 255, 195, 255,...","[969.4044950451057, 950.3049342295409, 951.568...","[2496.8758119491476, 2475.6812435272295, 2504....","[1846.9796474377404, 1856.2089048068233, 1813....","[4.369868755340576, 3.980140209197998, 3.33032...","[1411.1709378560383, 1840.4599278767903, 763.0...","[37.565555204948566, 42.90058190603934, 27.623...","[16.635007369993268, 15.481539948262977, 19.30...","[0.7749172731539066, 0.7689622899782756, 0.760...","[0.9611095053813692, 0.9518165916885702, 0.969..."
3,AirplaneF16.tiff,5,otsu,max,"[42, 93, 136, 174, 202]","[22, 68, 116, 161, 200]","[43, 99, 141, 174, 200]",964.461436,2546.930958,1862.135317,...,"[[99, 255, 200, 255, 255, 255, 200, 200, 200, ...","[945.942185713382, 953.7103045965292, 956.3143...","[2474.477458499088, 2551.689152036257, 2570.64...","[1842.8753046333754, 1849.3474587490728, 1836....","[4.287339210510254, 3.405366897583008, 4.37902...","[1786.3742663065593, 1410.5078150431316, 774.4...","[42.26552101070752, 37.55672795975618, 27.8290...","[15.611079070078194, 16.63704864122923, 19.240...","[0.7483039995752336, 0.780628465154723, 0.7204...","[0.9530445710759294, 0.9626790731657162, 0.973..."
4,Lena.png,2,otsu,max,"[136, 190]","[71, 138]","[97, 148]",922.194625,2223.838789,2108.303242,...,"[[148, 148, 148, 148, 148, 148, 148, 148, 148,...","[992.5117574694788, 943.8503060898321, 978.973...","[2265.0223696726343, 1896.5496498740133, 2356....","[2104.4509382643073, 2177.8577726093886, 2151....","[3.3528127670288086, 4.096403360366821, 3.1866...","[3286.1475003560386, 3584.73419825236, 2100.49...","[57.324929135202936, 59.872649834898404, 45.83...","[12.96393307808723, 12.58623401843693, 14.9075...","[0.606987831974012, 0.5661227157488873, 0.6072...","[0.875580024483598, 0.8479455566183297, 0.8861..."
5,Lena.png,3,otsu,max,"[112, 161, 208]","[45, 110, 172]","[67, 124, 167]",970.054141,2381.002847,2195.854004,...,"[[167, 167, 167, 167, 124, 124, 124, 167, 167,...","[989.6141896551233, 965.0221179106369, 981.299...","[2373.3522343839177, 2536.3404631740896, 2405....","[2250.566579146062, 2228.0747965490427, 2235.4...","[4.044633388519287, 4.205077648162842, 3.29329...","[1591.5327262878418, 1471.2197672526042, 1414....","[39.8940186780906, 38.35648272785976, 37.61087...","[16.11264787530685, 16.45402809434616, 16.6245...","[0.6413488360233296,

# Save results

In [8]:
pickle.dump(gwo_evaluation_results_df, open('results/evaluation/SMA/SMA_otsu_RGB_df.pkl', 'wb'))
pickle.dump(gwo_evaluation_results, open('results/evaluation/SMA/SMA_otsu_RGB_dict.pkl', 'wb'))